In [1]:
#include <sstream>
#include <vector>
#include <string>
#include <cmath>
#include <math.h>
#include <cstdlib>
#include <map>
#include "TH1.h"
#include "TH2.h"
#include "TProfile.h"
#include "TCanvas.h"
#include "TGraph.h"
#include "TFile.h"
#include "TString.h"
#include "TStyle.h"

using namespace std;

EXPLAIN ABOUT INTERPOLATION

In [2]:
double  interpolateTimewalk( TH1D* htw, int Npointsperbin, double &crossingphaseup, double &crossingphasedown){

    TString name = ""; name+=htw->GetName(); name+=+"_int";
    TString title = name+";interpolated phase; interpolated BXID";

    int npoints = Npointsperbin;
    int nbins = htw->GetNbinsX();
    TH1D interpoltw(name,"; BXID + phase", npoints*nbins, htw->GetBinLowEdge(1), htw->GetBinLowEdge(nbins+1));
    double precision = 10;
    double maxpoint = htw->GetMaximum();
    double edgepoint = maxpoint*0.50; //  rise and fall edge at 30% of max. 
    double lastintery = 0;
    crossingphaseup = crossingphaseup = 0;

    for (int interpoint = 0; interpoint<Npointsperbin*htw->GetNbinsX(); interpoint++){
        double interx = interpoltw.GetBinCenter(interpoint+1);
        double intery = htw->Interpolate(interx);
        double delta = (intery-lastintery);
        if(fabs(intery-edgepoint)< precision){
            if(delta>0) crossingphaseup = interx;
            if(delta<0) crossingphasedown = interx;
        }
        if (fabs(delta) < precision/100.) interpoint+=10;
            lastintery = intery;
    }

    cout<< " crossingphaseup = " << crossingphaseup << "  crossingphasedown "  << crossingphasedown << endl;
    cout << "edgepoint = " << edgepoint << endl;

    return 0.5*(crossingphaseup + crossingphasedown);

}

In [3]:
%%cpp
char myfname[100];
char myhname[100];

TFile *inputfile= TFile::Open("../data/outputfile_negative.root");
sprintf(myfname,"htwInterpPix.root");
TFile *outfile = new TFile(myfname, "RECREATE");
char title[65536];

for(int irow=3; irow<5; irow++){
    
    int icol = 0;
    TH1F *hlowEdge=NULL, *haverageEdge=NULL, *hhighEdge=NULL;
    TGraph *tgr=NULL;
    TH2F *htw=NULL;
    
    sprintf(myhname,"htwAllperpix_r%d_c%d", irow, icol);
    htw =  (TH2F*) inputfile->Get(myhname);

    // Check if histograms from input file exist
    if (htw==NULL){
        cout << " Where is " << myhname << " ? " << endl;
        return -1;
    }

    sprintf(title,"Crossing phase average at 30 percent - Pixel(%d,%d); #Delta VTP; bxid+(1.0/16)*phase", irow, icol);
    //sprintf(title,"Crossing phase r%d c%d", irow, icol);

    int nbins = htw->GetNbinsX();
    TString myname =""; myname+=htw->GetName(); myname+="_lowEdge";
    hlowEdge = new TH1F(myname.Data(),
                        "Crossing phase up; #Delta VTP; bxid+(1.0/16)*phase",
                        htw->GetNbinsX(),
                        htw->GetXaxis()->GetBinLowEdge(1),
                        htw->GetXaxis()->GetBinLowEdge(nbins+1));

    myname =""; myname+=htw->GetName(); myname+="_averageEdge";
    haverageEdge= new TH1F(myname.Data(), "Crossing phase up; #Delta VTP; bxid+(1.0/16)*phase", htw->GetNbinsX(), htw->GetXaxis()->GetBinLowEdge(1), htw->GetXaxis()->GetBinLowEdge(nbins+1));

    myname =""; myname+=htw->GetName(); myname+="_highEdge";
    hhighEdge = new TH1F(myname.Data(), title, htw->GetNbinsX(), htw->GetXaxis()->GetBinLowEdge(1), htw->GetXaxis()->GetBinLowEdge(nbins+1));


    vector<double> averages;
    vector<double>  mydeltas;
    for (int idelta = 1; idelta<513; idelta++){

        cout << "DeltaVTP-1 = " << idelta << endl << endl;

        TString myname2 =""; myname2 += htw->GetName(); myname2 +="_";myname2 +=idelta;

        cout << " myname  = " << myname2 << endl;
        cout << " myname.Data  = " << myname2.Data() << endl;

        TH1D * tempproj= (TH1D*) htw->ProjectionY(myname2.Data(), idelta, idelta);
        double cpup = 0, cpdown = 0;
        interpolateTimewalk(tempproj, 200, cpup, cpdown);

        if( cpup >0.5 && cpdown >0.5){
            averages.push_back((cpup+cpdown)*0.5);
            mydeltas.push_back(idelta);
            hlowEdge->SetBinContent(idelta, cpup);
            hhighEdge->SetBinContent(idelta, cpdown);
            haverageEdge->SetBinContent(idelta,(cpup+cpdown)*0.5);
        }

        delete tempproj;

    } //idelta

    double *myaverages_array = &averages[0]; //y
    double *mydeltas_array = &mydeltas[0];  //x
    int  Nvals = averages.size();

    tgr = new TGraph(Nvals,  mydeltas_array, myaverages_array);

    TString finalname = "finalplot_r";
    TString myfinalplot=""; myfinalplot+=finalname; myfinalplot+=Form("%d",(irow));
    myfinalplot+="_c"; myfinalplot+=Form("%d",(icol));

    cout << endl << " myfinalplot.Data  = " << myfinalplot.Data() << endl;
    if (tgr == NULL){cout << " ???? " << endl; return -2000;}
    tgr->SetName(myfinalplot.Data());
    tgr->SetTitle(title);
    tgr->Draw("ALP");

    gStyle->SetOptStat(0);
    hhighEdge->Draw("histp");
    hhighEdge->SetMarkerColor(1);
    hhighEdge->SetMarkerSize(0.7);
    hhighEdge->SetMarkerStyle(20);

    haverageEdge->Draw("histpsame");
    haverageEdge->SetMarkerColor(2);
    haverageEdge->SetMarkerSize(0.7);
    haverageEdge->SetMarkerStyle(20);

    hlowEdge->Draw("histpsame");
    hlowEdge->SetMarkerColor(1);
    hlowEdge->SetMarkerSize(0.7);
    hlowEdge->SetMarkerStyle(20);
    //outfile->Close();
    tgr->Write();
    haverageEdge->Write();
    hhighEdge->Write();
    hlowEdge->Write();
    htw->Write();
    
}


outfile->Close();


DeltaVTP-1 = 1

 myname  = htwAllperpix_r3_c0_1
 myname.Data  = htwAllperpix_r3_c0_1
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 2

 myname  = htwAllperpix_r3_c0_2
 myname.Data  = htwAllperpix_r3_c0_2
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 3

 myname  = htwAllperpix_r3_c0_3
 myname.Data  = htwAllperpix_r3_c0_3
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 4

 myname  = htwAllperpix_r3_c0_4
 myname.Data  = htwAllperpix_r3_c0_4
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 5

 myname  = htwAllperpix_r3_c0_5
 myname.Data  = htwAllperpix_r3_c0_5
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 6

 myname  = htwAllperpix_r3_c0_6
 myname.Data  = htwAllperpix_r3_c0_6
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 7

 myname  = htwAllperpix_r3_c0_7
 myname.Data  = htwAllperpix_r3_c0_7
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 

 crossingphaseup = 613.145  crossingphasedown 638.145
edgepoint = 110
DeltaVTP-1 = 70

 myname  = htwAllperpix_r3_c0_70
 myname.Data  = htwAllperpix_r3_c0_70
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 71

 myname  = htwAllperpix_r3_c0_71
 myname.Data  = htwAllperpix_r3_c0_71
 crossingphaseup = 612.629  crossingphasedown 637.629
edgepoint = 110
DeltaVTP-1 = 72

 myname  = htwAllperpix_r3_c0_72
 myname.Data  = htwAllperpix_r3_c0_72
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 73

 myname  = htwAllperpix_r3_c0_73
 myname.Data  = htwAllperpix_r3_c0_73
 crossingphaseup = 612.613  crossingphasedown 637.613
edgepoint = 110
DeltaVTP-1 = 74

 myname  = htwAllperpix_r3_c0_74
 myname.Data  = htwAllperpix_r3_c0_74
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 75

 myname  = htwAllperpix_r3_c0_75
 myname.Data  = htwAllperpix_r3_c0_75
 crossingphaseup = 612.176  crossingphasedown 637.176
edgepoint = 110
DeltaVTP-1 = 76

 myname

 crossingphaseup = 611.684  crossingphasedown 636.684
edgepoint = 110
DeltaVTP-1 = 140

 myname  = htwAllperpix_r3_c0_140
 myname.Data  = htwAllperpix_r3_c0_140
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 141

 myname  = htwAllperpix_r3_c0_141
 myname.Data  = htwAllperpix_r3_c0_141
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 142

 myname  = htwAllperpix_r3_c0_142
 myname.Data  = htwAllperpix_r3_c0_142
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 143

 myname  = htwAllperpix_r3_c0_143
 myname.Data  = htwAllperpix_r3_c0_143
 crossingphaseup = 611.629  crossingphasedown 636.629
edgepoint = 110
DeltaVTP-1 = 144

 myname  = htwAllperpix_r3_c0_144
 myname.Data  = htwAllperpix_r3_c0_144
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 145

 myname  = htwAllperpix_r3_c0_145
 myname.Data  = htwAllperpix_r3_c0_145
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 146

 myname  = htwAl

 crossingphaseup = 610.059  crossingphasedown 635.059
edgepoint = 110
DeltaVTP-1 = 212

 myname  = htwAllperpix_r3_c0_212
 myname.Data  = htwAllperpix_r3_c0_212
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 213

 myname  = htwAllperpix_r3_c0_213
 myname.Data  = htwAllperpix_r3_c0_213
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 214

 myname  = htwAllperpix_r3_c0_214
 myname.Data  = htwAllperpix_r3_c0_214
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 215

 myname  = htwAllperpix_r3_c0_215
 myname.Data  = htwAllperpix_r3_c0_215
 crossingphaseup = 610.066  crossingphasedown 635.027
edgepoint = 110
DeltaVTP-1 = 216

 myname  = htwAllperpix_r3_c0_216
 myname.Data  = htwAllperpix_r3_c0_216
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 217

 myname  = htwAllperpix_r3_c0_217
 myname.Data  = htwAllperpix_r3_c0_217
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 218

 myname  = htwAl

 crossingphaseup = 608.918  crossingphasedown 633.855
edgepoint = 110
DeltaVTP-1 = 284

 myname  = htwAllperpix_r3_c0_284
 myname.Data  = htwAllperpix_r3_c0_284
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 285

 myname  = htwAllperpix_r3_c0_285
 myname.Data  = htwAllperpix_r3_c0_285
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 286

 myname  = htwAllperpix_r3_c0_286
 myname.Data  = htwAllperpix_r3_c0_286
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 287

 myname  = htwAllperpix_r3_c0_287
 myname.Data  = htwAllperpix_r3_c0_287
 crossingphaseup = 608.848  crossingphasedown 633.848
edgepoint = 110
DeltaVTP-1 = 288

 myname  = htwAllperpix_r3_c0_288
 myname.Data  = htwAllperpix_r3_c0_288
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 289

 myname  = htwAllperpix_r3_c0_289
 myname.Data  = htwAllperpix_r3_c0_289
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 290

 myname  = htwAl

...

 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 225

 myname  = htwAllperpix_r4_c0_225
 myname.Data  = htwAllperpix_r4_c0_225
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 226

 myname  = htwAllperpix_r4_c0_226
 myname.Data  = htwAllperpix_r4_c0_226
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 227

 myname  = htwAllperpix_r4_c0_227
 myname.Data  = htwAllperpix_r4_c0_227
 crossingphaseup = 610.246  crossingphasedown 635.246
edgepoint = 110
DeltaVTP-1 = 228

 myname  = htwAllperpix_r4_c0_228
 myname.Data  = htwAllperpix_r4_c0_228
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 229

 myname  = htwAllperpix_r4_c0_229
 myname.Data  = htwAllperpix_r4_c0_229
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 230

 myname  = htwAllperpix_r4_c0_230
 myname.Data  = htwAllperpix_r4_c0_230
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 231

 myname  = htwAllperpix_r4_c0_

 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 295

 myname  = htwAllperpix_r4_c0_295
 myname.Data  = htwAllperpix_r4_c0_295
 crossingphaseup = 609.715  crossingphasedown 634.715
edgepoint = 110
DeltaVTP-1 = 296

 myname  = htwAllperpix_r4_c0_296
 myname.Data  = htwAllperpix_r4_c0_296
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 297

 myname  = htwAllperpix_r4_c0_297
 myname.Data  = htwAllperpix_r4_c0_297
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 298

 myname  = htwAllperpix_r4_c0_298
 myname.Data  = htwAllperpix_r4_c0_298
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 299

 myname  = htwAllperpix_r4_c0_299
 myname.Data  = htwAllperpix_r4_c0_299
 crossingphaseup = 609.629  crossingphasedown 634.629
edgepoint = 110
DeltaVTP-1 = 300

 myname  = htwAllperpix_r4_c0_300
 myname.Data  = htwAllperpix_r4_c0_300
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 301

 myname  = htwAl

 crossingphaseup = 609.129  crossingphasedown 634.129
edgepoint = 110
DeltaVTP-1 = 364

 myname  = htwAllperpix_r4_c0_364
 myname.Data  = htwAllperpix_r4_c0_364
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 365

 myname  = htwAllperpix_r4_c0_365
 myname.Data  = htwAllperpix_r4_c0_365
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 366

 myname  = htwAllperpix_r4_c0_366
 myname.Data  = htwAllperpix_r4_c0_366
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 367

 myname  = htwAllperpix_r4_c0_367
 myname.Data  = htwAllperpix_r4_c0_367
 crossingphaseup = 609.066  crossingphasedown 634.066
edgepoint = 110
DeltaVTP-1 = 368

 myname  = htwAllperpix_r4_c0_368
 myname.Data  = htwAllperpix_r4_c0_368
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 369

 myname  = htwAllperpix_r4_c0_369
 myname.Data  = htwAllperpix_r4_c0_369
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 370

 myname  = htwAl

 crossingphaseup = 608.934  crossingphasedown 633.871
edgepoint = 110
DeltaVTP-1 = 436

 myname  = htwAllperpix_r4_c0_436
 myname.Data  = htwAllperpix_r4_c0_436
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 437

 myname  = htwAllperpix_r4_c0_437
 myname.Data  = htwAllperpix_r4_c0_437
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 438

 myname  = htwAllperpix_r4_c0_438
 myname.Data  = htwAllperpix_r4_c0_438
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 439

 myname  = htwAllperpix_r4_c0_439
 myname.Data  = htwAllperpix_r4_c0_439
 crossingphaseup = 608.918  crossingphasedown 633.887
edgepoint = 110
DeltaVTP-1 = 440

 myname  = htwAllperpix_r4_c0_440
 myname.Data  = htwAllperpix_r4_c0_440
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 441

 myname  = htwAllperpix_r4_c0_441
 myname.Data  = htwAllperpix_r4_c0_441
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 442

 myname  = htwAl

 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 509

 myname  = htwAllperpix_r4_c0_509
 myname.Data  = htwAllperpix_r4_c0_509
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 510

 myname  = htwAllperpix_r4_c0_510
 myname.Data  = htwAllperpix_r4_c0_510
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0
DeltaVTP-1 = 511

 myname  = htwAllperpix_r4_c0_511
 myname.Data  = htwAllperpix_r4_c0_511
 crossingphaseup = 608.793  crossingphasedown 633.793
edgepoint = 110
DeltaVTP-1 = 512

 myname  = htwAllperpix_r4_c0_512
 myname.Data  = htwAllperpix_r4_c0_512
 crossingphaseup = 0  crossingphasedown 0
edgepoint = 0

 myfinalplot.Data  = finalplot_r4_c0
